In [ ]:
from pathlib import Path
import utils as utl
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

path_name = './flywire_ids_630.xlsx'

# sheet names of the xls file to include
sheets_pair = [ # pair of neurons (left+right per row)
    # 'stop',
    # 'walk',
    'walk_outputs',
    # 'SEZ',
    ]
sheets_single = [ # single neurons (one per row)
    # 'sugar', 
    # 'ovidn', 
    # 'bitter', 
    # 'water', 
    # 'jo_e', 
    # 'jo_f', 
    # 'eye_bristle', 
    ]

name2flyid = utl.create_name_dict(path_name, path_comp, sheets_pair, sheets_single)

In [ ]:
# load raw data, save rates
root = Path('./results/coac2/sens-walk/')

combination = 'sugarr+p9'

fs = [ *range(20, 210, 20) ]

ps = []
for f1 in fs:
    for f2 in fs:
        n = f'{combination}_{f1}_{f2}.parquet'
        ps.append(root / n)

df_spkt = utl.load_exps(ps)
df_rate, df_std = utl.get_rate(df_spkt, duration=1)
df_rate.to_parquet(f'rate_{combination}.parquet')


combination = 'sugarr+bpn'
ps = []
for f1 in fs:
    for f2 in fs:
        n = f'{combination}_{f1}_{f2}.parquet'
        ps.append(root / n)

df_spkt = utl.load_exps(ps)
df_rate, df_std = utl.get_rate(df_spkt, duration=1)
df_rate.to_parquet(f'rate_{combination}.parquet')

In [ ]:
# construct table for target neuron
target = 'P9-oDN1_r'

combination = 'sugarr+p9'

df_rate = pd.read_parquet(f'rate_{combination}.parquet')
df_rate = utl.rename_index(df_rate, name2flyid)

ds = df_rate.loc[target, :]
fs1 = [ int(i.split('_')[1]) for i in ds.index ]
fs2 = [ int(i.split('_')[2]) for i in ds.index ]

df = pd.pivot_table(
    data=pd.DataFrame(data={
        'r': ds.values,
        'f1': fs1, # sugar activation
        'f2': fs2
}), values='r', index='f2', columns='f1') # sugar becomes columns

df = df.sort_index(ascending=False)

df1 = df

combination = 'sugarr+bpn'

df_rate = pd.read_parquet(f'rate_{combination}.parquet')
df_rate = utl.rename_index(df_rate, name2flyid)

ds = df_rate.loc[target, :]
fs1 = [ int(i.split('_')[1]) for i in ds.index ]
fs2 = [ int(i.split('_')[2]) for i in ds.index ]

df = pd.pivot_table(
    data=pd.DataFrame(data={
        'r': ds.values,
        'f1': fs1, # sugar activation
        'f2': fs2
}), values='r', index='f2', columns='f1') # sugar becomes columns

df = df.sort_index(ascending=False)

df2 = df

In [ ]:
# plot

fig, axarr = plt.subplots(ncols=2, figsize=(10, 5))

vmax = max(df1.values.max(), df2.values.max()) # set to None to not match scale
cmap = 'viridis'
cbar_label = f'{target} firing rate [Hz]'

ax = axarr[0]
sns.heatmap(ax=ax, data=df1, cmap=cmap, vmin=0, vmax=vmax, cbar_kws={'label': cbar_label})
ax.set_ylabel('P9 activation [Hz]')

ax = axarr[1]
sns.heatmap(ax=ax, data=df2, cmap=cmap, vmin=0, vmax=vmax, cbar_kws={'label': cbar_label})
ax.set_ylabel('BPN activation [Hz]')

for ax in axarr:
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    ax.set_xlabel('sugar activation [Hz]')

fig.tight_layout()
fig.savefig('heatmap_2freq.svg')